In [55]:
import torch
import torch.nn as nn # NN networks (CNN, RNN, losses)
import torch.optim as optim # Aptimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F # Activarions func (ReLU, Sigmoid) also included in nn
import torchvision.datasets as datasets # Datasets
import torchvision.transforms as transforms # Transformation to datasets
import torchvision
import pandas as pd
from torch.utils.data import DataLoader, Dataset # Dataset manager
from torch.nn.utils.rnn import pad_sequence
from torchvision.utils import save_image
from PIL import Image
from skimage import io 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [56]:
class TextNoTextData(Dataset):
    def __init__(self, root_dir, labels_dir, transform=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.labels = pd.read_csv(labels_dir)
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.labels.iloc[index, 0])
        img = io.imread(img_path)
        y_label = torch.tensor(int(self.labels.iloc[index, 1]))
        if self.transform:
            img = self.transform(img)
            
        return img, y_label
        

In [57]:
root_dir = "./text_notext_competition/TrainData/all_data/"
labels_dir = "./text_notext_competition/train_labels.csv"
transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
])

dataset = TextNoTextData(root_dir=root_dir, labels_dir=labels_dir, transform=transformation)

In [58]:
dataset.__len__()

9608

In [59]:
batch_size = 128
train_amount = 7500
test_amount = dataset.__len__() - train_amount

train_set, test_set = torch.utils.data.random_split(dataset, [train_amount, test_amount])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)